In [1]:
import yfinance as yf

# Fetch historical data for a stock
def get_stock_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data

# Example usage
stock_data = get_stock_data("AAPL", "2020-01-01", "2023-01-01")


[*********************100%***********************]  1 of 1 completed


In [15]:
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler

# Extract 'Close' prices and scale them
prices = stock_data['Close'].values
scaler = MinMaxScaler()
prices_scaled = scaler.fit_transform(prices.reshape(-1, 1))

# Create sequences
def create_sequences(data, sequence_length):
    sequences = []
    labels = []
    for i in range(len(data) - sequence_length):
        seq = data[i:i + sequence_length]
        label = data[i + sequence_length]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

sequence_length = 60  # Use past 60 days to predict the next day
X, y = create_sequences(prices_scaled, sequence_length)

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

# Split the data into training and test sets
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


In [16]:
import torch.nn as nn

class StockLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(StockLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # Take the last time step
        return out

# Define hyperparameters
input_size = 1  # One feature: the scaled closing price
hidden_size = 50
num_layers = 2
output_size = 1

model = StockLSTM(input_size, hidden_size, num_layers, output_size)


In [17]:
import torch.optim as optim

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device ='mps'
model.to(device)

# Train the model
epochs = 50
batch_size = 32

for epoch in range(epochs):
    for i in range(0, len(X_train) - batch_size, batch_size):
        model.train()
        X_batch = X_train[i:i + batch_size]
        y_batch = y_train[i:i + batch_size]
        
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # Evaluate the model every 10 steps
    if (epoch+1) % 10 == 0:
        model.eval()
        for i in range(0, len(X_train) - batch_size, batch_size):
            X_batch = X_test[i:i + batch_size]
            y_batch = y_test[i:i + batch_size]
            
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            
            with torch.no_grad():
                test_outputs = model(X_batch)
                test_loss = criterion(test_outputs, y_batch)
                            
        print(f"Epoch {epoch+1}/{epochs}, Step {i+1}, Loss: {loss.item()}, Test Loss: {test_loss.item()}")
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")


Epoch 1/50, Loss: 0.16517913341522217
Epoch 2/50, Loss: 0.005898762494325638
Epoch 3/50, Loss: 0.038983315229415894
Epoch 4/50, Loss: 0.006563826464116573
Epoch 5/50, Loss: 0.007742389105260372
Epoch 6/50, Loss: 0.0022179323714226484
Epoch 7/50, Loss: 0.02224283292889595
Epoch 8/50, Loss: 0.011004285886883736
Epoch 9/50, Loss: 0.02002818137407303
Epoch 10/50, Loss: 0.00987694226205349
Epoch 11/50, Loss: 0.010115423239767551
Epoch 12/50, Loss: 0.005637315101921558
Epoch 13/50, Loss: 0.0062082745134830475
Epoch 14/50, Loss: 0.013067007064819336
Epoch 15/50, Loss: 0.011462759226560593
Epoch 16/50, Loss: 0.009545507840812206
Epoch 17/50, Loss: 0.005688107572495937
Epoch 18/50, Loss: 0.004683839622884989
Epoch 19/50, Loss: 0.006372132338583469
Epoch 20/50, Loss: 0.0077201006934046745
Epoch 21/50, Loss: 0.007927716709673405
Epoch 22/50, Loss: 0.007496410980820656
Epoch 23/50, Loss: 0.007052105385810137
Epoch 24/50, Loss: 0.006835492327809334
Epoch 25/50, Loss: 0.006744640879333019
Epoch 26/5

In [13]:
model.eval()
model.to('cpu')
with torch.no_grad():
    test_inputs = X[-batch_size:]
    predictions = model(test_inputs).numpy()

# Reverse scaling
predictions_unscaled = scaler.inverse_transform(predictions)


In [14]:
predictions_unscaled


array([[149.54237],
       [150.64673],
       [151.57402],
       [152.51363],
       [153.39249],
       [153.80415],
       [154.18846],
       [154.58589],
       [154.63943],
       [154.15024],
       [153.16917],
       [152.77246],
       [152.6784 ],
       [152.70021],
       [152.66266],
       [152.2294 ],
       [151.47531],
       [150.82962],
       [150.24213],
       [149.9916 ],
       [150.01859],
       [149.94492],
       [149.16277],
       [147.95703],
       [146.45572],
       [144.9692 ],
       [143.96577],
       [142.96173],
       [142.05023],
       [141.09149],
       [139.8092 ],
       [138.87886]], dtype=float32)